In [1]:
from datasets import load_from_disk
from transformers import AutoTokenizer
import numpy as np
from tqdm import tqdm, trange
import argparse
import random
import torch
import torch.nn.functional as F
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)

from transformers import BertModel, BertPreTrainedModel, AdamW, TrainingArguments, get_linear_schedule_with_warmup

In [2]:
torch.manual_seed(2021)
torch.cuda.manual_seed(2021)
np.random.seed(2021)
random.seed(2021)

In [3]:
datasets=load_from_disk("../data/train_dataset/")
train_dataset,valid_dataset=datasets['train'],datasets['validation']

model_name="klue/bert-base"
tokenizer=AutoTokenizer.from_pretrained(model_name)

In [4]:
q_seqs = tokenizer(train_dataset['question'], padding="max_length", truncation=True, return_tensors='pt')
p_seqs = tokenizer(train_dataset['context'], padding="max_length", truncation=True, return_tensors='pt')
train_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'], p_seqs['token_type_ids'], 
                        q_seqs['input_ids'], q_seqs['attention_mask'], q_seqs['token_type_ids'])

In [5]:
q_seqs = tokenizer(valid_dataset['question'], padding="max_length", truncation=True, return_tensors='pt')
p_seqs = tokenizer(valid_dataset['context'], padding="max_length", truncation=True, return_tensors='pt')
valid_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'], p_seqs['token_type_ids'],
                                q_seqs['input_ids'], q_seqs['attention_mask'], q_seqs['token_type_ids'])

In [37]:
class BertEncoder(BertPreTrainedModel):
  def __init__(self, config):
    super(BertEncoder, self).__init__(config)

    self.bert = BertModel(config)
    self.init_weights()
      
  def forward(self, input_ids, 
              attention_mask=None, token_type_ids=None): 
  
      outputs = self.bert(input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids)
      
      pooled_output = outputs[1]

      return pooled_output

In [6]:
class BertAvgEncoder(BertPreTrainedModel):
    def __init__(self, config):
        super(BertAvgEncoder,self).__init__(config)
        self.bert=BertModel(config)
        self.init_weights()
    
    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs=self.bert(input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids)
        valid_length=torch.sum(attention_mask,dim=-1)
        valid_length=valid_length.unsqueeze(dim=-1)

        sum_hidden=torch.sum(outputs[0],dim=1)
        #sum_hidden=torch.sum(outputs.last_hidden_state,dim=1)
        avg_output=sum_hidden/valid_length

        return avg_output

In [7]:
p_encoder=BertAvgEncoder.from_pretrained(model_name)
q_encoder=BertAvgEncoder.from_pretrained(model_name)
if torch.cuda.is_available():
    p_encoder.cuda()
    q_encoder.cuda()

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertAvgEncoder: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertAvgEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertAvgEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertAvgEncoder: ['cls.pr

In [8]:
no_decay = ["bias" ,"LayerNorm.weight"]
optimizer_grouped_parameters = [
    {"params": [p for n, p in p_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": 0.01},
    {"params": [p for n, p in p_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    {"params": [p for n, p in q_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": 0.01},
    {"params": [p for n, p in q_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0}
]
optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=5e-5,
    # eps=args.adam_epsilon
)

In [9]:
train_loader = DataLoader(train_dataset,batch_size=20)
valid_loader=DataLoader(valid_dataset,batch_size=20)

In [10]:
def proprecessing(text):
    new_text = text.replace(r'\n\n','')
    return new_text

In [11]:
def dense_embedding(dataset):
    """문맥의 임베딩 값을 구하고리턴합니다."""
    q_seqs = tokenizer(dataset['question'], padding="max_length", truncation=True, return_tensors='pt')
    p_seqs = tokenizer(dataset['context'], padding="max_length", truncation=True, return_tensors='pt')
    # print(q_seqs[0])
    dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'], p_seqs['token_type_ids'],
                                    q_seqs['input_ids'], q_seqs['attention_mask'], q_seqs['token_type_ids'])
    dataloader = DataLoader(dataset,batch_size=20)
    print("Build passage embedding")
    for idx, data in enumerate(tqdm(dataloader)):
        p_inputs = {'input_ids': data[0].to('cuda'),
                    'attention_mask': data[1].to('cuda'),
                    'token_type_ids': data[2].to('cuda')
                    }
        with torch.no_grad():
            p_inputs = {k: v for k, v in p_inputs.items()}
            output = p_encoder(**p_inputs)
            if idx == 0:
                p_embedding = output
            else:
                p_embedding = torch.cat((p_embedding, output), 0)
    return p_embedding

In [12]:
best = 0
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
train_dataset,valid_dataset=datasets['train'],datasets['validation']

train_datasets = {}
train_datasets['context'] = [proprecessing(string) for string in train_dataset['context']]
train_datasets['question'] = train_dataset['question']
valid_datasets = {}
valid_datasets['context'] = [proprecessing(string) for string in valid_dataset['context']]
valid_datasets['question'] = valid_dataset['question']
for epoch in range(5):
    step = 0
    losses = 0
    for idx,data in enumerate(tqdm(train_loader)):
        step += 1
        p_inputs = {'input_ids': data[0].to(device),
                    'attention_mask': data[1].to(device),
                    'token_type_ids': data[2].to(device)
                    }
        q_inputs = {'input_ids': data[3].to(device),
                    'attention_mask': data[4].to(device),
                    'token_type_ids': data[5].to(device)}
        targets = torch.arange(0, len(p_inputs['input_ids'])).long().to(device)
        q_output = q_encoder(**q_inputs)
        p_output = p_encoder(**p_inputs)
        retrieval = torch.matmul(q_output,p_output.T)
        retrieval_scores = F.log_softmax(retrieval, dim=1)

        loss = F.nll_loss(retrieval_scores, targets)
        losses += loss.item()
        # q_encoder.zero_grad()
        p_encoder.zero_grad()
        q_encoder.zero_grad()

        loss.backward()
        optimizer.step()
        # scheduler.step()
        if step % 100 == 0:
            print(f'{epoch}epoch loss: {losses/(step)}')

            losses = 0
            correct = 0
            step = 0
    optimizer.lr =5e-6
    valid_embedding = dense_embedding(valid_datasets)
    valid_ans = []
    for idx,data in enumerate(tqdm(valid_loader)):
        with torch.no_grad():
            q_inputs = {'input_ids': data[3].to(device),
                        'attention_mask': data[4].to(device),
                        'token_type_ids': data[5].to(device)}
            output = p_encoder(**q_inputs)
            if idx == 0:
                query_vec = output
            else:
                query_vec = torch.cat((query_vec, output), 0)

    result = torch.mm(query_vec, valid_embedding.T)
    result = result.cpu().detach().numpy()
    doc_indices = []
    for i in range(result.shape[0]):
        sorted_result = np.argsort(result[i, :])[::-1]
        doc_indices.append(sorted_result.tolist()[:3])
    correct = 0
    for idx,i in enumerate(doc_indices):
        if idx in i:
            correct += 1
    print(f'acc :{correct/len(valid_dataset)}')
print('save')
torch.save(p_encoder.state_dict(), 'gold/p_encoder.pt')
torch.save(q_encoder.state_dict(), 'gold/q_encoder.pt')
p_embedding = dense_embedding(train_datasets)

 51%|█████     | 100/198 [02:00<01:58,  1.21s/it]

0epoch loss: 143.64504294395448


  0%|          | 0/12 [00:00<?, ?it/s]

Build passage embedding


  0%|          | 0/198 [00:00<?, ?it/s]

...

100%|██████████| 12/12 [00:02<00:00,  5.01it/s]


acc :0.10416666666666667
save


  0%|          | 0/198 [00:00<?, ?it/s]

Build passage embedding


100%|██████████| 198/198 [00:39<00:00,  5.00it/s]


In [16]:
datasets['validation']['question']

['처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?',
 '스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?',
 '촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?',
 '로타이르가 백조를 구하기 위해 사용한 것은?',
 '의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?',
 '1945년 쇼와천황의 항복 선언이 발표된 라디오 방송은?',
 '징금수는 서양 자수의 어떤 기법과 같은 기술을 사용하는가?',
 '다른 과 의사들은 감염내과 전문의들로부터 어떤 것에 대해 조언을 받는가?',
 '루이 14세의 왕비 마리아 테래사는 어느 나라 공주인가?',
 '헤자즈 왕국이 실존했던 것은 언제까지인가?',
 '버드 교장이 5월의 여왕의 대안으로 제시한 것은?',
 "인형사'를 만들어낸 것으로 추측되는 사업의 이름은?",
 '멘데스가 요원들을 구하기 위해 간 도시는 어디인가?',
 '교과부의 행동에 화가나 여러명이 사직한 기구의 이름은?',
 '반대동맹이 공산당과 갈라서겠다고 얘기한 날은 언제인가?',
 '피어슨이 다시 의회를 해산했던 년도는?',
 '몽케가 죽은 뒤 쿠릴타이에서 대칸의 지위를 얻은 사람의 이름은?',
 '이흥구의 사법시험 이야기를 기사로 작성한 곳은?',
 '남북조 시대에서 이이 씨가 전쟁이 발생했을 때, 생활했던 장소는?',
 '박지훈은 1라운드에서 몇 순위를 차지했는가?',
 '데메카론에는 무엇을 풍자하는 이야기가 들어있나요?',
 '병에 걸려 죽을 확률이 약 25~50%에 달하는 유형의 질병은?',
 '설리반이 불만을 표시한 대상은 누구인가?',
 '베소스는 어디서 추방당했는가?',
 '진전사의 명칭이 드러나는 데 영향을 준 물건은?',
 '자신의 이상적인 국가관이 스파르타와 닮아 있다고 생각하는 플라톤의 저서는?',
 '박제된 북극곰이 사망한 날짜는?',
 '문법 측면에서 더 보수적인 포르투갈어 표준은?',
 '로스 수장이 살해한 사람은 어느 당 회원인가?',
 '조경숙왕의 아들인 요자의 친어머니는 누구